In [ ]:
dataset_location = "/tmp/clevr-act-1"
print("Train set:")
!head -n 5 {dataset_location}/dataset/_annotations.train.jsonl
print("\nValidation set:")
!head -n 5 {dataset_location}/dataset/_annotations.valid.jsonl

In [ ]:
import cv2
import json
import supervision as sv
from typing import List
import re

def read_n_lines(file_path: str, n: int) -> List[str]:
    with open(file_path, 'r') as file:
        lines = [next(file).strip() for _ in range(n)]
    return lines

def parse_trajectory_tokens(caption):
  caption = caption.strip("\n")
  if ";" in caption:
    raise ValueError
  pattern = r"(?:<loc\d{4}>)+ ([\w\s\-]+)$"
  match = re.search(pattern, caption)
  #print(f"Matched words: {match.group(1)}")
  if match:
    return match
  else:
    return ""

images = []
lines = read_n_lines(f"{dataset_location}/dataset/_annotations.train.jsonl", 25)
first = json.loads(lines[0])

#CLASSES = first.get('prefix').replace("detect ", "").split(" ; ")
for line in lines:
    data = json.loads(line)
    #image = cv2.imread(f"{dataset_location}/dataset/{data.get('image')}")
    #(h, w, _) = image.shape

    suffix = data.get('suffix')
    prefix = data.get('prefix')
    #match = parse_trajectory_tokens(suffix)
    print(prefix)
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import html
import base64
from IPython.display import display, HTML


text = "<loc0625><loc0762><loc0468><loc0724><loc0384><loc0679><loc0375><loc0632><loc0430><loc0587><loc0587> 1"
action_text = data.get('prefix')

def render_example_trajectory(image, caption):
    # Pattern to extract numbers inside <loc####> tags
    loc_strings = re.findall(r"<loc(\d{4})>", caption)
    num_position_tokens = len(loc_strings)
    loc_strings_pairs = loc_strings[:(num_position_tokens//2)*2]
    loc_numbers = [int(x) for x in loc_strings_pairs]
    loc_h = [x/1024*image_height for x in loc_numbers[::2]]
    loc_w = [x/1024*image_width for x in loc_numbers[1::2]]
    curve_2d_short = curve_2d = np.array([np.stack((loc_w, loc_h), axis=1)])

    env_id = 0
    x, y = curve_2d[env_id, :, 0].tolist(), curve_2d[env_id, :, 1].tolist()

    pixel_width, pixel_height = 448, 448
    dpi = 100
    figsize = (pixel_width / dpi, pixel_height / dpi)
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.imshow(image)
    ax.axis('off')
    ax.plot(curve_2d_short[env_id,:,0], curve_2d_short[env_id, :,1],'.-', color='lime')
    with io.BytesIO() as buffer:
        fig.savefig(buffer, format='jpeg',bbox_inches='tight', dpi=dpi)
        image_b64 = str(base64.b64encode(buffer.getvalue()), "utf-8")
        res_str =  f"data:image/jpeg;base64,{image_b64}"
    plt.close(fig)
    return f"""
<div style="display: inline-flex; align-items: center; justify-content: center;">
    <img style="width:224px; height:224px;" src="{res_str}" />
    <p style="width:256px; margin:10px; font-size:small;">{html.escape(caption)}</br>{html.escape(caption)}</p>

</div>
"""

html_out = ""
#for image, _, caption in make_predictions(validation_data_iterator(), num_examples=1, batch_size=1):
#  html_out += render_example(image, caption)

caption = data.get('prefix')
image = Image.open(f"{dataset_location}/dataset/{data.get('image')}")
html_out += render_example_trajectory(image, caption)
display(HTML(html_out))
print("done!")

In [ ]:
matches[:, 0]